<a href="https://colab.research.google.com/github/zahidhamidi/ML-Project/blob/main/Unit_test_RBM_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import re
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from tqdm.auto import tqdm
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_excel("/content/sample_data/robust_test_dataset.xlsx")

In [23]:
def text_extraction(text,topic):

  # Define the topics as flexible terms (subwords or whole words)
  # topics = ["gas shows" , "oil show", "overpressure", "fluid communication" , "gas bearing" , "h2s" , "co2" , "stuck pipe" , "pressure build up", "sand production"]
  topics = ["gas-bearing"]

  # Replace hyphens with blank spaces in all items in the topics list
  topics = [topic.replace('-', ' ') for topic in topics]

  # Define the negation terms pattern with case-insensitivity
  negation_terms = r'(?i)(?:(?<=\s)|^)(no|non|unlikely|none|not|nor|without|lack|rather)(?=\s|$|\b)'

  # Initialize a list to store selected sentences
  selected_sentences = []

  # Initialize a flag to check for both topic terms and negation terms
  both_detected = False

  # Tokenize the text into sentences using NLTK's sent_tokenize
  sentences = sent_tokenize(text)

  # Initialize flags to check for both topic terms and negation terms
  topic_found = False
  negation_detected = False

  for sentence in sentences:

    # remove apostrophes from string
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", " ")

    # Tokenize the sentence into words
    lemmatized_words = word_tokenize(sentence)

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = ' '.join(lemmatized_words)

    # Initialize an empty pattern string
    pattern_string = r'(?i)(?:(?<=\s)|^)'

    for topic_pattern in topics:
      words = topic_pattern.split()

      if len(words) >= 2:

        # Split the topic words by spaces and hyphens and create a pattern to match them
        topic_pattern = re.compile(r'(?i)\b' + r'\b|\b'.join(re.escape(topic) for topic in topics) + r'\b')

        if re.search(topic_pattern, lemmatized_sentence.lower()):
              topic_found = True
              break

      elif len(words) != 0:
          single_pattern = rf'(?i)(?:(?<=\s)|^)({re.escape(words[0])})(?=\s|$)'
          if re.search(single_pattern, lemmatized_sentence.lower()):
              topic_found = True
              break

    # Check if the lemmatized sentence contains negation terms
    if re.search(negation_terms, lemmatized_sentence.lower()):
        negation_detected = True
    else:
        negation_detected = False

    counter = 0
    # If a topic term is found and either no negation terms are found or negation_detected is False, append the sentence
    if (topic_found is True) and (negation_detected is True):
        selected_sentences.append(lemmatized_sentence)
        both_detected = True
        break



    elif (topic_found is True) and (negation_detected is False):
        selected_sentences.append(lemmatized_sentence)
        both_detected = False
        counter = counter + 1

        if counter >1:
          break
        break


    # If no sentence with both topic and negation terms is found, append the first sentence without a negation term
    # if not both_detected or topic_found is False:
    #     selected_sentences.append(lemmatized_sentence)

  # print("Selected Sentence: ", selected_sentences)
  # print("Topic word found: ",topic_found)
  # print("Negation word found: ",negation_detected)
  return selected_sentences








In [24]:
text = '535 mMD RKB, as they are correlated to gas bearing sands in other wells. They did not, however, display seismic anomalies at the well site. '
selected = text_extraction(text,"gas bearing")
selected

['535 mMD RKB as they are correlated to gas bearing sands in other wells .']

In [15]:
selected_text = []

for i in tqdm(range(len(df))):
  for doc in df["doc_text_original"]:
    topic = df.iloc[i,39]
    selected = text_extraction(doc,topic)
    selected_text.append(selected)



  0%|          | 0/2161 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [12]:
 # Add the selected_sentences as a new column named 'selected_sentence'
# df['selected_sentence'] = selected_text
len(selected_text)

4669921